In [2]:
import os
os.getcwd()
os.chdir("c:\\Users\\neilp\\Documents\\College\\F20PA\\f20pa-2023-24-student-revision-system")

In [36]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()

# loader = TextLoader("alice_in_wonderland.txt", encoding="utf-8")
loader = UnstructuredFileLoader("09_week8_neural_networks.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name


In [3]:
from langchain_community.document_loaders.directory import DirectoryLoader

loader = DirectoryLoader("lectures")
docs = loader.load()

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

In [5]:
chunks[7]

Document(page_content='Reduces dimensionality & feature size (can help with overfitting in ML models)\n\nIntroduction to Text Analytics\n\n15\n\nFinance\n\nD2\n\nD4\n\nD3\n\nSports\n\nD5\n\nD1\n\nEducation\n\nVector space model\n\nCosine similarity\n\nJaccard Similarity\n\nEach document is represented as a vector. All documents are projected into the space.\n\nOne dimension for every unique term in space(axes)\n\n|D2-D4|\n\nHigh-dimensional vector space: 10,000+ dimensions, or even 100,000+\n\nIntroduction to Text Analytics\n\n16\n\nBag of words example\n\nMost simple but effective and commonly used ways to represent text for machine learning\n\nFor applications such as classification, topic-modeling, and recommender systems, this type of representation is sufficient.\n\nIntroduction to Text Analytics\n\n17\n\nDocument Vectorization\n\nDocument is represented with a numerical vector \n\nDimension of the vector =distinct terms in ”corpus”', metadata={'source': 'lectures\\01_week1_intro_

In [11]:
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()
model = ChatOpenAI(model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q5_K_M.gguf", base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)

vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever()

In [7]:
vectorstore.similarity_search("common activation functions", k=1)

[Document(page_content='Activation Functions\n\nLinear Activation functions produce linear decisions no matter the network size\n\nNon-linearities allow us to approximate arbitrarily complex functions\n\nCommon Activation Functions\n\nSigmoid Function ge) wr 0.8 ge) A 06 Pa F 0.4 / 0.2 oe ——" 5 0 5 (@)=— Zz) = ————_ g 1+e2 g\' (2) = g(a - g(2)) tf£.nn.sigmoid(z) Hyperbolic Tangent g(2) g\'@) Z_ eZ I@)= ay e* g\'(z) = 1-g(z)? NOTE: All activation functions are non-linear Rectified Linear Unit (ReLU) 5 / at) 4 9) 3 / 2 rd 1 ~-— 0 i 5 ie} 5 g (Zz) = max(0,z) 1 z>0 Ul —_— , g\'(2) = {3 otherwise tf.nn.relu(z)\n\nDATA\n\nWhich dataset do you want to use?\n\n\n\nRatio of training to test data: 50%\n\n—®\n\nNoise:\n\n0\n\nBatch size:\n\n10\n\n—e\n\nREGENERATE\n\nEpoch\n\nLearning rate\n\nActivation\n\nRegularization\n\n000,000\n\n0.03\n\nTanh\n\n,\n\nhttp://playground.tensorflow.org/\n\nFEATURES\n\n+ —- 2 HIDDEN LAYERS\n\nWhich properties do you want to feed in?\n\n+\n\n+ -', metadata={'sourc

In [12]:
from langchain_core.prompts import PromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context","question"]
  )

In [13]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs,  "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [14]:
# question = "Explain the difference between batch gradient descent, stochastic gradient descent, and mini-batch gradient descent in terms of the number of samples processed before updating the model parameters."
query = "What are some common activation functions?"
rag_chain.invoke(query)

' Some common activation functions include the Sigmoid Function and the Hyperbolic Tangent function, as mentioned in the context. Additionally, the Rectified Linear Unit (ReLU) function is also commonly used. The equations for these functions are provided in the context.'

In [15]:
rag_chain.invoke("Who is Neil Armstrong?")

' Based on the context I cannot answer that question as there is no information provided about Neil Armstrong in the context given.'

In [32]:
rag_chain.invoke("What are neural networks?")

'\nAnswer: Neural networks are a type of artificial intelligence model inspired by the structure and function of the human brain. They consist of interconnected processing nodes, or neurons, that work together to identify patterns in data. Neural networks can learn from data through a process called training, during which they adjust their internal connections based on feedback from the data. Deep learning is a type of neural network that involves multiple layers of representations, allowing for more complex pattern recognition and feature extraction. The universal approximation theorem states that any continuous function can be approximated arbitrarily well by a neural network with at least one hidden layer, but in practice, deeper networks with many layers may be required to effectively learn and generalize from data. Neural networks use activation functions to introduce non-linearity and allow for the modeling of complex relationships between inputs and outputs. Common activation fu